In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer


In [11]:
movies = pd.read_csv("movies.csv")

genres = movies["genres"].str.split("|")
genres = genres.apply(lambda x: " ".join(x))


In [12]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(genres)

In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
movie_to_idx = {title: idx for idx, title in enumerate(movies["title"])}

In [15]:
def get_recommendation(movie_title, top_N=10):
    movie_idx = movie_to_idx.get(movie_title, None)

    if movie_idx is None:
        return []
    
    sim_scores = list(enumerate(cosine_sim[movie_idx]))

    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

    tope_N_movies = [movies["title"][i] for i, _ in sim_scores[1:top_N+1]]
    
    return tope_N_movies

In [16]:
movies_input = widgets.Text(placeholder = "Enter a movie title", description="Movie:")
top_N_input = widgets.IntSlider(min=1, max=50, step=1, description="Top N:")

In [17]:
def update_recommendations(change):
    movie_title = movies_input.value
    top_N = top_N_input.value

    recommendations = get_recommendation(movie_title, top_N)

    output.clear_output()
    with output:
        if recommendations:
            print(f"Top {top_N} Recommended Movies for '{movie_title}':")
            for i, movie in enumerate(recommendations):
                print(f"{i+1}. {movie}")
        else:
            print("Movie not found or not enough data for recommendation")

In [18]:
search_button = widgets.Button(description="Get Recommendations")
search_button.on_click(update_recommendations)

In [20]:
output = widgets.Output()
layout = widgets.VBox([movies_input, top_N_input, search_button, output])
display(layout)